Importing the dataset

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
yelp_data = pd.read_csv('/content/yelp_labelled.txt', sep='\t', header=None)

In [5]:
yelp_data.head()

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [6]:
column_name = ['Review', 'Sentiment']
yelp_data.columns = column_name

In [7]:
yelp_data.tail(2)

,Review,Sentiment
998,"The whole experience was underwhelming, and I ...",0
999,"Then, as if I hadn't wasted enough of my life ...",0


In [8]:
yelp_data.shape

(1000, 2)

In [9]:
amazon_data = pd.read_csv('/content/amazon_cells_labelled.txt', sep='\t', header=None)

In [10]:
amazon_data.head()

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [11]:
column_name = ['Review', 'Sentiment']
amazon_data.columns = column_name

In [12]:
amazon_data.tail(2)

,Review,Sentiment
998,The only thing that disappoint me is the infra...,0
999,"You can not answer calls with the unit, never ...",0


In [13]:
amazon_data.shape

(1000, 2)

In [14]:
imdb_data = pd.read_csv('/content/imdb_labelled.txt', sep='\t', header=None)

In [15]:
imdb_data.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [16]:
column_name = ['Review', 'Sentiment']
imdb_data.columns = column_name

In [17]:
imdb_data.tail(2)

,Review,Sentiment
746,Exceptionally bad!,0
747,All in all its an insult to one's intelligence...,0


In [18]:
imdb_data.shape

(748, 2)

In [19]:
# Merge all dataset in a dataframe
data = pd.concat([yelp_data, amazon_data, imdb_data], axis = 0, ignore_index=True)

In [20]:
data.shape

(2748, 2)

In [21]:
data.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [22]:
# Count of each type of sentiment
data['Sentiment'].value_counts()

1    1386
0    1362
Name: Sentiment, dtype: int64

In [23]:
# Count of null values
data.isnull().sum()

Review       0
Sentiment    0
dtype: int64

In [24]:
X = data['Review']
y = data['Sentiment']

Data Cleaning

In [25]:
import string
import spacy

In [26]:
punc = string.punctuation

In [27]:
punc

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [28]:
from spacy.lang.en.stop_words import STOP_WORDS

In [29]:
stop_words = list(STOP_WORDS)

In [30]:
stop_words[:5]

['we', 'everywhere', 'before', 'whether', 'put']

In [31]:
len(stop_words)

326

In [32]:
# Function for data cleaning
def data_cleaning(sentence):
  nlp =  spacy.load('en_core_web_sm')
  doc = nlp(sentence)

  tokens = [] # list of tokens
  for token in doc:
    # if root isn't Pronoun
    if token.lemma_ != "-PRON-":
      # convert into lower form
      temp = token.lemma_.lower().strip()
    else:
      temp = token.lower_
    tokens.append(temp)

  cleaned_token = []
  for token in tokens:
    if token not in stop_words and token not in punc:
      cleaned_token.append(token)

  return cleaned_token


In [33]:
data_cleaning(data["Review"][4])

['selection', 'menu', 'great', 'price']

In [34]:
data_cleaning("Hello all, It's a beautiful day outside there!")

['hello', 'beautiful', 'day', 'outside']

Vectorizer Feature Engineering (TF-IDF)

In [35]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [36]:
tfidf = TfidfVectorizer(tokenizer=data_cleaning)

In [37]:
classifier = LinearSVC()

Training Model

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

In [39]:
X_train.shape, y_train.shape

((2198,), (2198,))

In [41]:
X_test.shape, y_test.shape

((550,), (550,))

In [42]:
X_train.head()

2572    An Italian reviewer called this "a small, grea...
526                          And it was way to expensive.
1509    As an earlier review noted, plug in this charg...
144     Nice blanket of moz over top but i feel like t...
2483    The film gives meaning to the phrase, "Never i...
Name: Review, dtype: object

Fit train & test data

In [43]:
clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])

In [44]:
clf.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function data_cleaning at 0x7b058acba0e0>)),
                ('clf', LinearSVC())])

Predicting test result

In [45]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [46]:
y_pred = clf.predict(X_test)

In [47]:
confusion_matrix(y_test, y_pred)

array([[201,  78],
       [ 50, 221]])

In [48]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.72      0.76       279
           1       0.74      0.82      0.78       271

    accuracy                           0.77       550
   macro avg       0.77      0.77      0.77       550
weighted avg       0.77      0.77      0.77       550



In [49]:
accuracy_score(y_test, y_pred)

0.7672727272727272

In [50]:
clf.predict(["Wow, I'm learning Natural Language Processing in fun fashion!"])

array([1])

In [51]:
clf.predict(["It's hard to learn new things!"])

array([0])